# Setup

In [2]:
!pip install nltk

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download(['stopwords', 'punkt'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kinga/snap/jupyter/6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kinga/snap/jupyter/6/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Input reader

In [26]:
import random

import pandas as pd
import xlrd
import csv
from sklearn import preprocessing


def read_csv(file_name):
    data = pd.read_csv(file_name)
    return data


def read_excel(file_name, sheet_name):
    data = pd.read_excel(file_name, sheet_name=sheet_name, index_col=None, usecols=['message', 'purpose'])
    return data.to_dict(orient='record')


def xlsx_to_csv(xlsx_filename):
    xlsx_file = xlrd.open_workbook(xlsx_filename)
    sheet = xlsx_file.sheet_by_name('Sheet1')
    csv_name = xlsx_filename.replace('xslx', 'csv')
    csv_file = open(csv_name, 'w')
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)

    for row_num in range(sheet.nrows):
        wr.writerow(sheet.row_values(row_num))
    csv_file.close()

    return csv_name


def split_set(data_set, index, seed):
    random.seed(seed)
    random.shuffle(data_set)

    split_index = int(index*len(data_set))
    return data_set[:split_index], data_set[split_index:]


def encode_labels(y):
    return preprocessing.LabelEncoder().fit_transform(y)


def generate_indices(length, index, seed):
    random.seed(seed)
    np_array = np.arange(len(data_set))
    random.shuffle(np_array)
    split_index = int(index*len(data_set))
    return np_array[:split_index], np_array[split_index:]


# Bag of words

In [4]:
import heapq

import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer


def preprocess_data(dataset):
    tokenizer = RegexpTokenizer("[a-zA-Z@]+")
    stemmer = SnowballStemmer("english")
    stop = stopwords.words('english')
    excluding = ['against', 'not', 'don', "don't", 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
                 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
                 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shouldn', "shouldn't", 'wasn',
                 "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    stop = [words for words in stop if words not in excluding]

    for index, message in enumerate(dataset):
        message = tokenizer.tokenize(message)
        dataset[index] = " ".join([stemmer.stem(w) for w in message if w not in stop])
    return dataset


def count_words_frequencies(dataset):
    frequencies = {}
    for message in dataset:
        words = nltk.word_tokenize(message)
        for word in words:
            if word not in frequencies.keys():
                frequencies[word] = 1
            else:
                frequencies[word] += 1
    return frequencies


def create_vector(messages, frequent_words):
    bow_model = []
    for message in messages:
        vector = []
        for word in frequent_words:
            if word in nltk.word_tokenize(message):
                vector.append(1)
            else:
                vector.append(0)
        bow_model.append(vector)
    return np.asarray(bow_model)


def create_bow_model(data_set_x, frequent_words_count):
    data_set_x = preprocess_data(data_set_x)
    word_frequencies = count_words_frequencies(data_set_x)
    frequent_words = heapq.nlargest(frequent_words_count, word_frequencies, key=word_frequencies.get)
    bow_model = create_vector(data_set_x, frequent_words)
    return bow_model


# Displaying results of GridSearch

In [5]:
''' source https://www.kaggle.com/grfiv4/displaying-the-results-of-a-grid-search '''

def GridSearch_table_plot(grid_clf, param_name,
                          num_results=15,
                          negative=True,
                          display_all_params=True):

    '''Display grid search results

    Arguments
    ---------

    grid_clf           the estimator resulting from a grid search
                       for example: grid_clf = GridSearchCV( ...

    param_name         a string with the name of the parameter being tested

    num_results        an integer indicating the number of results to display
                       Default: 15

    negative           boolean: should the sign of the score be reversed?
                       scoring = 'neg_log_loss', for instance
                       Default: True

    display_all_params boolean: should we print out all of the parameters, not just the ones searched for?
                       Default: True

    Usage
    -----

    GridSearch_table_plot(grid_clf, "min_samples_leaf")

                          '''
    from matplotlib      import pyplot as plt
    from IPython.display import display
    import pandas as pd

    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')

    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    # =====================================
    display(pd.DataFrame(cv_results) \
            .sort_values(by='rank_test_score').head(num_results))

    # plot the results
    # ================
    scores_df = scores_df.sort_values(by='param_' + param_name)

    if negative:
        means = -scores_df['mean_test_score']
    else:
        means = scores_df['mean_test_score']
    stds = scores_df['std_test_score']
    params = scores_df['param_' + param_name]


# Classifiers

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


def measure_efficiency(y, predictions):
    print('Accuracy Score : ', str(accuracy_score(y, predictions)))


class Classifier:
    def __init__(self, seed, train_bow_model, train_set_y, test_bow_model, test_set_y):
        self.seed = seed
        self.train_bow_model = train_bow_model
        self.train_set_y = train_set_y
        self.test_bow_model = test_bow_model
        self.test_set_y = test_set_y

    def logistic_regression(self):
        print('Logistic regression')
        log_reg = LogisticRegression(random_state=self.seed, max_iter=1000000, solver='liblinear', C=0.1)
        log_reg.fit(self.train_bow_model, self.train_set_y)
        predictions = log_reg.predict(self.test_bow_model)
        measure_efficiency(self.test_set_y, predictions)

    def grid_search_logistic_regression(self):
        param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                      'solver': ['liblinear', 'newton-cg', 'sag', 'saga', 'lbfgs']}
        grid_search = GridSearchCV(LogisticRegression(random_state=self.seed, max_iter=1000000), param_grid, verbose=1)
        grid_search.fit(self.train_bow_model, self.train_set_y)
        predictions = grid_search.predict(self.test_bow_model)
        measure_efficiency(self.test_set_y, predictions)
        GridSearch_table_plot(grid_search, "C", graph=False, negative=False)

    def random_forest(self, estimators_number):
        print('Random forest')
        rfc = RandomForestClassifier(estimators_number, random_state=self.seed)
        rfc.fit(self.train_bow_model, self.train_set_y)
        predictions = rfc.predict(self.test_bow_model)
        measure_efficiency(self.test_set_y, predictions)

    def grid_search_random_forest(self):
        param_grid = {'n_estimators': [1, 10, 100, 1000]}
        grid_search = GridSearchCV(RandomForestClassifier(random_state=self.seed), param_grid, verbose=1)
        grid_search.fit(self.train_bow_model, self.train_set_y)
        predictions = grid_search.predict(self.test_bow_model)
        measure_efficiency(self.test_set_y, predictions)
        GridSearch_table_plot(grid_search, "n_estimators", graph=False, negative=False)

    def decision_tree(self):
        print('Decision tree')
        decision_tree = DecisionTreeClassifier(random_state=self.seed)
        decision_tree.fit(self.train_bow_model, self.train_set_y)
        predictions = decision_tree.predict(self.test_bow_model)
        measure_efficiency(self.test_set_y, predictions)


# Experiment

### Generate indexes to files

In [49]:
train_indexes, test_indexes = generate_indices(len(data_set), 0.9, seed)
indexes = [train_indexes, test_indexes]

with open("indexes.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(indexes)

In [7]:
gerrit_file = 'gerrit-wireshark-train-test-v4.xlsx'
seed = 40


data_set = read_excel(gerrit_file, 'train')

data_set_x = list([row['message'] for row in data_set])
data_set_y = list([row['purpose'] for row in data_set])

train_set_x, test_set_x = split_set(data_set_x, 0.9, seed)
train_set_y, test_set_y = split_set(data_set_y, 0.9, seed)

frequent_words_count = 100
train_bow_model = create_bow_model(train_set_x, frequent_words_count)
test_bow_model = create_bow_model(test_set_x, frequent_words_count)

classifier = Classifier(seed, train_bow_model, train_set_y, test_bow_model, test_set_y)

classifier.logistic_regression()
classifier.random_forest(100)
classifier.decision_tree()

Logistic regression
Accuracy Score :  0.64
Random forest
Accuracy Score :  0.62
Decision tree
Accuracy Score :  0.52
